In [1]:
# 🛠️ Install Transformers and Torch if not already installed
!pip install transformers --quiet
!pip install torch --quiet

# 🧠 Import Core Libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
import re
from tqdm import tqdm
import os

print("✅ Packages installed and imported.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.6 MB/s eta 0:00:00
✅ Packages installed and imported.


In [2]:
# 🌍 World Profile (Trimmed for GPT-2 Medium)
world_profile = """
Two years ago, the Suntear Lance destroyed the Kingdom of Aristo, leaving the Hollow Grounds — a cursed crater of wild magic and broken lands. Survivors struggle to rebuild while hidden factions battle for control. Strange arcane forces stir beneath the ruins, and old loyalties are tested in a world reshaped by catastrophe.
"""

first_mission_intro = """
📜 Whispers Over McGovin 📜

The town of McGovin struggles under smoky skies, broken remnants of Aristo littering the land.

Suddenly — a shout. Across the square, dark-cloaked figures clash with desperate rebels: the Thorn Pact fighting to survive against Obsidian Court conspirators.

At the caravan’s center stands **Wailer**, a tactical hobgoblin commander known for dry wit, and **He'Jock**, a grizzled HAMMER dwarf tinkering with a rune-etched crossbow.

The battle threatens to consume McGovin.

🏹 Will you aid the Thorn Pact?
🧠 Or stay hidden and learn the truth of this shadow war?
"""



In [3]:
# 🧠 Load the GPT-2 Medium Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"✅ Model loaded on {device}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded on cpu


In [4]:
# 🎯 Simple Training Dataset
# (This is very basic — we'll later improve with richer player/DM examples if needed)

# Example player-DM exchanges for training
training_data = [

    # Basic narrative moves
    {"input": "Player: I watch from the shadows.\nDM:", "output": "Hidden in the ruins, you see cloaked figures rallying their forces, oblivious to your presence."},
    {"input": "Player: I search the ruined buildings for supplies.\nDM:", "output": "Dust and rubble choke the old halls, but a careful search yields a rusty sword and a pouch of coins."},
    {"input": "Player: I call out to the rebels.\nDM:", "output": "The rebels turn, wary but desperate, their weapons half-raised in suspicion and hope."},
    {"input": "Player: I study the cloaked men from a distance.\nDM:", "output": "Their black sigils shimmer faintly — a mark of the Obsidian Court, if the old tales are true."},
    {"input": "Player: I try to remember old legends about this place.\nDM:", "output": "Whispers of ancient battles and cursed relics fill your mind, faint echoes of the Hollow's dark history."},
        # Sneaking toward enemies
    {"input": "Player: I try to sneak closer to the enemy lines.\nDM:", "output": "LOW: A boot scrapes stone. One of the cloaked figures glances sharply in your direction."},
    {"input": "Player: I try to sneak closer to the enemy lines.\nDM:", "output": "MEDIUM: You move between the broken walls, narrowly avoiding a patrolling rebel's gaze."},
    {"input": "Player: I try to sneak closer to the enemy lines.\nDM:", "output": "HIGH: You glide like a shadow, unnoticed, reaching a prime vantage point behind enemy lines."},

    # Attacking an enemy
    {"input": "Player: I strike at the cloaked figure.\nDM:", "output": "LOW: Your blade glances off the enemy's armor. He snarls and counters."},
    {"input": "Player: I strike at the cloaked figure.\nDM:", "output": "MEDIUM: You land a blow, driving the cloaked man back with a hiss of pain."},
    {"input": "Player: I strike at the cloaked figure.\nDM:", "output": "HIGH: Your strike is flawless, felling the cloaked figure in one decisive blow."},

    # Persuading a rebel
    {"input": "Player: I try to persuade the rebel to help us.\nDM:", "output": "LOW: The rebel scowls. 'You're not one of us,' he spits."},
    {"input": "Player: I try to persuade the rebel to help us.\nDM:", "output": "MEDIUM: The rebel hesitates, eyes darting. 'Maybe... if you prove yourself first.'"},
    {"input": "Player: I try to persuade the rebel to help us.\nDM:", "output": "HIGH: The rebel nods firmly. 'I'm with you. Tell me what you need.'"},

    # Searching the ruins
    {"input": "Player: I search the ruins for something useful.\nDM:", "output": "LOW: The debris shifts, revealing nothing but shattered wood and mold."},
    {"input": "Player: I search the ruins for something useful.\nDM:", "output": "MEDIUM: You uncover a battered shield and a handful of usable arrows."},
    {"input": "Player: I search the ruins for something useful.\nDM:", "output": "HIGH: Hidden in a crevice, you find an enchanted blade faintly pulsing with power."},

    # Rallying survivors
    {"input": "Player: I rally the survivors to stand and fight.\nDM:", "output": "LOW: Fear tightens their faces. No one moves."},
    {"input": "Player: I rally the survivors to stand and fight.\nDM:", "output": "MEDIUM: A few hesitant nods — some step forward to fight by your side."},
    {"input": "Player: I rally the survivors to stand and fight.\nDM:", "output": "HIGH: A cheer rises! Inspired, the survivors charge into battle with renewed fury."}
]
# Critical Success/Failure Focused Samples

training_data += [

    # Critical Success (Roll 20)
    {"input": "Player: I roll a 20 to strike the enemy.\nDM:", "output": "With blinding speed, you strike down two foes in a single sweeping arc, sending shockwaves across the battlefield!"},
    {"input": "Player: I roll a 20 to sneak past guards.\nDM:", "output": "You move so silently that even the wind seems to forget your passage; no guard stirs."},
    {"input": "Player: I roll a 20 to persuade the rebels.\nDM:", "output": "Your words ignite fire in their hearts — the rebels rally with renewed strength under your command."},
    {"input": "Player: I roll a 20 to dodge incoming attacks.\nDM:", "output": "You weave through a hail of arrows, untouched, moving with otherworldly grace."},
    {"input": "Player: I roll a 20 to search the ruins.\nDM:", "output": "Hidden deep within the rubble, you uncover an ancient relic, pulsing faintly with power."},

    # Critical Failure (Roll 1)
    {"input": "Player: I roll a 1 to strike the enemy.\nDM:", "output": "Your weapon slips from your grasp mid-swing, clattering noisily across the stone floor."},
    {"input": "Player: I roll a 1 to sneak past guards.\nDM:", "output": "You trip over loose rubble with a loud crash — all eyes turn toward you in alarm!"},
    {"input": "Player: I roll a 1 to persuade the rebels.\nDM:", "output": "Your words stumble and falter — the rebels scoff and turn their backs, suspicion heavy in the air."},
    {"input": "Player: I roll a 1 to dodge incoming attacks.\nDM:", "output": "You misstep terribly, falling hard — and an enemy’s blade catches your exposed side."},
    {"input": "Player: I roll a 1 to search the ruins.\nDM:", "output": "You dig frantically, dislodging a pile of unstable rubble that collapses around you in a choking cloud."},
]


# Convert to list of text samples
texts = []
for item in training_data:
    texts.append(item["input"] + " " + item["output"])

print(f"✅ Prepared {len(texts)} training samples.")


✅ Prepared 30 training samples.


In [5]:
# 🧠 Tokenize the Training Texts

encodings = tokenizer(
    texts,
    truncation=True,
    padding=True,
    return_tensors="pt",
    max_length=512  # GPT-2 Medium can handle up to around 1024 tokens, but we stay safe
)

print(f"✅ Tokenized {len(encodings['input_ids'])} entries for training.")


✅ Tokenized 30 entries for training.


In [6]:
# 🗂️ Create Dataset and DataLoader

class DMGameDataset(Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.input_ids[idx],  # GPT2 is causal — input and labels are the same
        }

# 📦 Initialize Dataset and DataLoader
dataset = DMGameDataset(encodings)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

print(f"✅ Dataset and DataLoader ready. Batch size = {loader.batch_size}")


✅ Dataset and DataLoader ready. Batch size = 1


In [ ]:
# 🛠️ Fine-Tuning Loop for GPT-2 Medium

from torch.optim import AdamW

# Optimizer Setup
optimizer = AdamW(model.parameters(), lr=5e-5)

# Fine-Tuning Mode
model.train()

# Number of Epochs
epochs = 3  # You can adjust (4–6 recommended)
batch_size = 1
for epoch in range(epochs):
    print(f"🔄 Epoch {epoch+1}/{epochs}")
    loop = tqdm(loader, leave=True)

    for batch in loop:
        batch = {k: v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

print("✅ Fine-tuning complete!")


🔄 Epoch 1/3


Epoch 0: 100%|██████████| 30/30 [02:50<00:00,  5.69s/it, loss=1.98]


🔄 Epoch 2/3


Epoch 1: 100%|██████████| 30/30 [02:35<00:00,  5.20s/it, loss=1.17]


🔄 Epoch 3/3


Epoch 2:  53%|█████▎    | 16/30 [01:33<01:23,  5.95s/it, loss=0.617]

In [ ]:
# 🧙‍♂️ Dungeon Master - Start Mission Interaction (Polished and Merged)
# 🧙‍♂️ Dungeon Master Start Mission (Start Mode + Active Session Mode)

story_so_far = ""

def update_story_summary(history):
    global story_so_far
    if len(history) <= 2:
        story_so_far = ""
    else:
        important_turns = "\n".join(history[-3:])  # Last few interactions only
        story_so_far = f"Session Summary:\n{important_turns}\n"

def roll_dice(notation):
    match = re.match(r"(\d*)d(\d+)", notation.lower())
    if match:
        count = int(match.group(1)) if match.group(1) else 1
        sides = int(match.group(2))
        rolls = [random.randint(1, sides) for _ in range(count)]
        return rolls, sum(rolls)
    return None, 0

def start_mission_interact():
    print("🧙‍♂️ Dungeon Master is ready. Type 'exit' to quit.\n")

    history = []
    first_turn = True

    while True:
        if first_turn:
            intro_text = f"{world_profile}\n\n{first_mission_intro}\n"
            print(f"📜 {first_mission_intro}\n")
            history.append(intro_text)
            first_turn = False

        player_input = input("Player: ")
        if player_input.lower() in {"exit", "quit"}:
            break

        if "roll" in player_input.lower():
            match = re.search(r"(\d*d\d+)", player_input.lower())
            if match:
                notation = match.group(1)
                rolls, total = roll_dice(notation)
                print(f"🎲 You rolled: {rolls} (Total: {total})\n")

                history.append(f"Player: I rolled a {total}")

                if total == 20:
                    history.append("(The player rolled a 20 — describe an epic heroic success.)")
                elif total == 1:
                    history.append("(The player rolled a 1 — describe a dramatic critical failure.)")
                else:
                    history.append("(The player rolled — narrate a normal success or failure.)")
                continue

        if re.search(r"rolled\s+\d+", player_input.lower()):
            history.append("(The player rolled — narrate a normal success or failure.)")

        history.append(f"Player: {player_input}")

        if len(history) > 6:
            history = history[-6:]  # Only keep 6 most recent moves

        update_story_summary(history)

        if len(history) <= 6:
            # 🧠 First few moves (after intro) - full prompt
            prompt = (
                f"{world_profile}\n\n"
                f"{first_mission_intro}\n\n"
                f"{story_so_far}\n\n"
                "Recent events:\n"
                + "\n".join(history[-4:]) +
                "\n\n"
                "(Narrate the next moment naturally. If the player rolled, describe success or failure based on the number. If a 20, describe a legendary success. If a 1, describe a tragic failure. Always move the story forward.)\n"
                "DM:"
            )
        else:
            # 🧠 After launch — active session mode
            prompt = (
                f"{story_so_far}\n\n"
                "Recent events:\n"
                + "\n".join(history[-4:]) +
                "\n\n"
                "(Narrate the next moment naturally. If the player rolled, describe success or failure based on the number. Always move the story forward.)\n"
                "DM:"
            )

        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)

        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=100,
            temperature=0.65,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        response = decoded.split("DM:")[-1].strip()

        if not response or "try again" in response.lower():
            response = "The winds howl across the Hollow Grounds. Time to act!"

        print(f"DM: {response}\n")
        history.append(f"DM: {response}")


In [ ]:
start_mission_interact()